**Shopify Data Science Intern Challenge 2022**

Submission by Alexandre Kishi.

In [15]:
#Initialization & importing data file
import os
import re 
import pandas as pd

Data = pd.read_csv("2019 Winter Data Science Intern Challenge Data Set.csv", sep = ',', header=0, engine='python')
Data #Data has been imported correctly as a DataFrame. 

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
0,1,53,746,224,2,cash,2017-03-13 12:36:56
1,2,92,925,90,1,cash,2017-03-03 17:38:52
2,3,44,861,144,1,cash,2017-03-14 4:23:56
3,4,18,935,156,1,credit_card,2017-03-26 12:43:37
4,5,18,883,156,1,credit_card,2017-03-01 4:35:11
...,...,...,...,...,...,...,...
4995,4996,73,993,330,2,debit,2017-03-30 13:47:17
4996,4997,48,789,234,2,cash,2017-03-16 20:36:16
4997,4998,56,867,351,3,cash,2017-03-19 5:42:42
4998,4999,60,825,354,2,credit_card,2017-03-16 14:51:18


In [16]:
Data.isnull().sum() #Check for any missing values

order_id          0
shop_id           0
user_id           0
order_amount      0
total_items       0
payment_method    0
created_at        0
dtype: int64

In [17]:
Data.describe() #Summary statistics

,order_id,shop_id,user_id,order_amount,total_items
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.00000
mean,2500.500000,50.078800,849.092400,3145.128000,8.78720
std,1443.520003,29.006118,87.798982,41282.539349,116.32032
min,1.000000,1.000000,607.000000,90.000000,1.00000
25%,1250.750000,24.000000,775.000000,163.000000,1.00000
50%,2500.500000,50.000000,849.000000,284.000000,2.00000
75%,3750.250000,75.000000,925.000000,390.000000,3.00000
max,5000.000000,100.000000,999.000000,704000.000000,2000.00000


The data has been loaded correctly and has no missing values. From the descriptive statistics, the AOV of $3145.13 was found by simply taking the average order value of all orders, for all shops. As this average is rather high for orders of shoes, this seems to indicate the presence of extreme values.

In [5]:
Data.sort_values(by=['order_amount'], ascending=False) #Sort by Order amount in Descending order

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
2153,2154,42,607,704000,2000,credit_card,2017-03-12 4:00:00
3332,3333,42,607,704000,2000,credit_card,2017-03-24 4:00:00
520,521,42,607,704000,2000,credit_card,2017-03-02 4:00:00
1602,1603,42,607,704000,2000,credit_card,2017-03-17 4:00:00
60,61,42,607,704000,2000,credit_card,2017-03-04 4:00:00
...,...,...,...,...,...,...,...
3871,3872,92,818,90,1,debit,2017-03-18 9:10:08
2503,2504,92,935,90,1,credit_card,2017-03-28 2:30:26
4414,4415,92,927,90,1,credit_card,2017-03-17 9:57:01
3363,3364,92,730,90,1,credit_card,2017-03-11 23:20:31


In [6]:
ShopAOV = Data.groupby(['shop_id']).mean() #Group into AOV by shop
ShopAOV = ShopAOV.drop(['order_id','user_id'], axis=1)
ShopAOV.columns = ['Avg order amount', 'Avg items per order']
ShopAOV.sort_values(by=['Avg order amount'], ascending=False)

,Avg order amount,Avg items per order
shop_id,,
42,235101.490196,667.901961
78,49213.043478,1.913043
50,403.545455,2.090909
90,403.224490,2.265306
38,390.857143,2.057143
...,...,...
53,214.117647,1.911765
100,213.675000,1.925000
32,189.976190,1.880952


A better approach would therefore be to find the AOV per shop, instead of all shops together. By doing so and sorting by average order, we can see that Shops 42 and 78 have a significantly higher AOV than the rest of the stores. The data from these two stores should then be investigated further. 

For shop 42, the AOV is 235k$, with an average of 668 items per order. This seems to indicate the presence of bulk orders, possibly from another business. 

For the shop 78, the AOV is 49k$, with an average of just under two items per order. This indicates small orders of items with extremely high prices. 

In [7]:
Shop42 = Data[Data["shop_id"] == 42]
Shop42

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
15,16,42,607,704000,2000,credit_card,2017-03-07 4:00:00
40,41,42,793,352,1,credit_card,2017-03-24 14:15:41
60,61,42,607,704000,2000,credit_card,2017-03-04 4:00:00
308,309,42,770,352,1,credit_card,2017-03-11 18:14:39
409,410,42,904,704,2,credit_card,2017-03-04 14:32:58
520,521,42,607,704000,2000,credit_card,2017-03-02 4:00:00
834,835,42,792,352,1,cash,2017-03-25 21:31:25
835,836,42,819,704,2,cash,2017-03-09 14:15:15
938,939,42,808,1056,3,credit_card,2017-03-13 23:43:45
979,980,42,744,352,1,debit,2017-03-12 13:09:04


Indeed, Shop42 has multiple repeat orders of 2000 items totalling 704k$ each time. These bulk orders heavily skew both the shop's and the collective AOV. 

For this exercise, we will exclude these orders from further analysis. In a real scenario, it could be relevant to examine these bulk orders in an independant analysis. 

In [8]:
Shop78 = Data[Data["shop_id"] == 78]
Shop78

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
160,161,78,990,25725,1,credit_card,2017-03-12 5:56:57
490,491,78,936,51450,2,debit,2017-03-26 17:08:19
493,494,78,983,51450,2,cash,2017-03-16 21:39:35
511,512,78,967,51450,2,cash,2017-03-09 7:23:14
617,618,78,760,51450,2,cash,2017-03-18 11:18:42
691,692,78,878,154350,6,debit,2017-03-27 22:51:43
1056,1057,78,800,25725,1,debit,2017-03-15 10:16:45
1193,1194,78,944,25725,1,debit,2017-03-16 16:38:26
1204,1205,78,970,25725,1,credit_card,2017-03-17 22:32:21
1259,1260,78,775,77175,3,credit_card,2017-03-27 9:27:20


For Shop78, it seems that every single item sold had a price of 25 725$. For a pair of shoes, this type of value seems unusually high and would require some validation. It is possible that the wrong price was entered by mistake, in which case validation followed by correction if required. Another possibility is that these are in fact extremely high value designer/exclusive shoes, in which case it would be relevant to exclude these orders and examine them independantly.

For this exercise, we will exclude these orders as we have no further information concerning them to validate with. 

In [9]:
Data_revised = Data[Data["order_amount"] < 25000] #Excluding both Shop42's bulk orders and Shop78's high value orders
ShopAOV_revised = Data_revised.groupby(['shop_id']).mean() #Group into AOV by shop using revised dataset
ShopAOV_revised = ShopAOV_revised.drop(['order_id','user_id'], axis=1)
ShopAOV_revised.columns = ['Avg order amount', 'Avg items per order']
ShopAOV_revised.sort_values(by=['Avg order amount'], ascending=False)

,Avg order amount,Avg items per order
shop_id,,
42,652.235294,1.852941
50,403.545455,2.090909
90,403.224490,2.265306
38,390.857143,2.057143
81,384.000000,2.169492
...,...,...
53,214.117647,1.911765
100,213.675000,1.925000
32,189.976190,1.880952


In [13]:
Data_revised.describe() 

,order_id,shop_id,user_id,order_amount,total_items
count,4937.000000,4937.000000,4937.000000,4937.000000,4937.000000
mean,2499.551347,49.846465,849.752279,302.580514,1.994734
std,1444.069407,29.061131,86.840313,160.804912,0.982821
min,1.000000,1.000000,700.000000,90.000000,1.000000
25%,1248.000000,24.000000,775.000000,163.000000,1.000000
50%,2497.000000,50.000000,850.000000,284.000000,2.000000
75%,3751.000000,74.000000,925.000000,387.000000,3.000000
max,5000.000000,100.000000,999.000000,1760.000000,8.000000


In [12]:
ShopAOV_revised.mean() #Finding the mean of all the shop AOV's

Avg order amount       303.243539
Avg items per order      1.993174
dtype: float64

Once excluding the outlying values, our results are much more reasonable for shops selling shoes. The naive AOV of all orders is now 302.58$, which is far more plausible.

A better metric for this dataset would be the individual shop AOV's, which would allow to examine the average sales for each individual shop. The mean of these AOV's can then also be taken, and is found to be 303.24$.

Further analysis of the shop AOV's is now possible and open to further investigation. 